In [ ]:
# default_exp masks

In [ ]:
from speechsep.imports import *
from speechsep.plot import *
from speechsep.utils import *

#hide
fn_list = Path("../data/SmallDataset/esc50_sample/").glob(r"*/*")
audio_list = [AudioBase(*load(fn)) for fn in fn_list]

# Sound Separation Masks
> All AudioMasking techniques that I know of.

In [ ]:
#export
class MaskBase():
    _show_args={}
    def __init__(self, data, spec_func):
        store_attr(self, 'data', 'spec_func')
    @property
    def shape(self):
        return self.data.shape
    @classmethod
    def create(cls, audios):
        joined = join_audios(audios)
        return [cls.generate(audio, joined) for audio in audios]
    def __mul__(self, spec):
        raise NotImplementedError('This function needs to be implemented before use')
    def __rmul__(self, spec):
        return self*spec
    def generate(self, audio, joined):
        raise NotImplementedError('This function needs to be implemented before use')
    @delegates(setup_graph)
    def show(self, ctx=None, **kwargs): return show_mask(self, ctx=ctx, **merge(self._show_args, kwargs))


In [ ]:
MaskBase._tensor_cls = TensorMask
@ToTensor
def encodes(self, o:MaskBase):  return o._tensor_cls(mask2tensor(o))
def mask2tensor(mask:MaskBase):  return TensorMask(mask.data)

In [ ]:
#export
class MaskBinary(MaskBase):
    def __mul__(self, spec):
        new_spec = SpecImage(spec.data*self.data, spec.sr, spec.fn)
        return new_spec
    @classmethod
    def generate(cls, audio, joined):
        specs = self.spec_func(audio)
        return (joined.data <= spec.data)*1

In [ ]:
#export
class MaskcIRM(MaskBase):
    def __mul__(self, spec):
        new_spec = SpecImage(spec.data*self.data, spec.sr, spec.fn)
        return new_spec
    @classmethod
    def generate(cls, audio, joined):
        specs = self.spec_func(audio)
        return (joined.data <= spec.data)*1